<a href="https://colab.research.google.com/github/solanods/vacation_ranking/blob/main/ranqueamento_ferias2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RANQUEAMENTO DE GOZO DE FÉRIAS EM MÊS NOBRE

Sistema de ranqueamento para gozo de férias em meses nobres: dez, jan, fev, mar.

Base de dados contendo os dados de férias dos operadores

Manter atualizada com base nos dados de férias dos últimos quatro anos.

LINK PARA DOWNLOAD DOS DADOS NO TOTVS

https://portalrm.trensurb.gov.br/Corpore.Net/Main.aspx?ActionID=BisCuboActionWeb&SelectedMenuIDKey=CubosNet&ShowMode=0&DetailIndex=-1&TransactionID=-1&KeyValues=1;149

O período para ranqueamento anual compreende as férias dos últimos quatro anos.

Exemplo 01/04/2019 a 31/03/2022  para ranqueamento 2023/2024

Exemplo 01/04/2020 a 31/03/2023  para ranqueamento 2024/2025

Ao acessar o link, alterar os parâmetros conforme acima.

OBS. É necessário salvar o arquivo em formato planilha do excel ".xlsx" antes do carregamento no Google Colab



In [25]:
# Imports
import pandas as pd
import numpy as np
from datetime import date
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings("ignore")

# Colunas alvo: RE, NOME, INICIO DAS FERIAS, DIAS DE FERIAS, DIAS DE ABONO
df = pd.read_excel('/content/FERIASMARCADASDOSEMPREGADOSPORTAL.xlsx',
                   usecols=[0,1,4,6,7])

# Preencher os valores ausentes
df = df.fillna(method='ffill')

# Criar mascara para eliminar as entradas que contenham a palavra Total no campo NOME
substring = 'Total'
mascara = df['NOME'].str.contains(substring)==False
df = df[mascara]

# Remover as linha duplicadas
df = df.drop_duplicates()

# Converter colunas RE e DIAS DE FÉRIAS para inteiro
df['RE'] = df['RE'].astype(int)
df['DIAS DE FÉRIAS'] = df['DIAS DE FÉRIAS'].astype(int)
df['DIAS DE ABONO'] = df['DIAS DE ABONO'].astype(int)

# Cria campo MES extraindo o mes
df['MES'] = df['INICIO DAS FERIAS'].dt.month

# Cria o campo ANO no tipo int
df['ANO'] = df['INICIO DAS FERIAS'].dt.year
df['ANO'] = df['ANO'].astype(int)

# Dividir os dados ateriores a 2023 para aplicação dos pesos antigos
df1 = df[df['INICIO DAS FERIAS'] <= pd.to_datetime('2022-12-31')]
df2 = df[df['INICIO DAS FERIAS'] >= pd.to_datetime('2023-01-01')]

# Criar um classificador para os períodos anteriores a 2023
condicao1 = [df1['MES'] == 1,
            df1['MES'] == 2,
            df1['MES'] == 12]

classe_mes = ['NOBRE', 'NOBRE', 'NOBRE']

df1['CLASSE MES'] = np.select(condicao1, classe_mes, 'NAO NOBRE')

# Criar pesos para os meses anteriores a 2023
peso_mes1 = [2, 2, 2]
df1['PESO'] = np.select(condicao1, peso_mes1, 1)

# Criar um classificador para os períodos a partir de 2023
condicao2 = [df2['MES'] == 1,
            df2['MES'] == 2,
            df2['MES'] == 3,
            df2['MES'] == 12]

classe_mes = ['NOBRE', 'NOBRE', 'NOBRE', 'NOBRE']
df2['CLASSE MES'] = np.select(condicao2, classe_mes, 'NAO NOBRE')

# Criar pesos para os meses a partir de 2023
peso_mes2 = [3, 3, 2, 2]
df2['PESO'] = np.select(condicao2, peso_mes2, 1)

# Juntar novamente os dados
dados = pd.concat([df1, df2])

# Ordenacao
dados = dados.sort_values(by=['RE', 'INICIO DAS FERIAS'], ignore_index=True)

# Calculo
dados['SUBTOTAL'] = (dados['DIAS DE FÉRIAS'] + dados['DIAS DE ABONO']) * dados['PESO']

# Criar a coluna de dias nobres
condicional = [dados['CLASSE MES'] == 'NOBRE',
               dados['CLASSE MES'] == 'NAO NOBRE']

subtotal_nobre = [dados['SUBTOTAL'], 0]

dados['DIAS NOBRES'] = np.select(condicional, subtotal_nobre, 0)

# Soma agrupada
pontuacao = dados.groupby(["RE"])["SUBTOTAL"].agg("sum")

# Soma dos dia de meses nobres
dias_nobre = dados.groupby(["RE"])["DIAS NOBRES"].agg("sum")

# Criar DataFrame
RE = []
for i in set(dados['RE']):
  RE.append(i)
RE.sort()

PONTUACAO = []
for i in pontuacao:
  PONTUACAO.append(int(i))

DIAS_NOBRES = []
for i in dias_nobre:
  DIAS_NOBRES.append(int(i))

ranking = pd.DataFrame({"RE": RE,
                       "PONTUACAO": PONTUACAO,
                       "DIAS NOBRES": DIAS_NOBRES})

resultado = ranking.sort_values(by=['PONTUACAO', 'DIAS NOBRES', 'RE'],
                                ignore_index=True)

# Enviar para o gdrive
now = date.today()
dados.to_excel(f'/content/drive/MyDrive/FERIAS/cálculos_ranking{now}.xlsx',
               sheet_name='DADOS_RANQUEAMENTO', index=False)

resultado.to_excel(f'/content/drive/MyDrive/FERIAS/classificacao_ranking{now}.xlsx',
                   sheet_name='DIAS_NOBRES', index=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
